In [1]:
from gurobipy import *
import numpy as np
import pandas as pd
import datetime as dt
import time
import glob
import random
import math

### Data Generation

In [2]:
Trbl_Trck_Hr_Rate = 250 #Dollars per Hour
Trbl_Shift_Duration = 12 #Hours
Penalty_Cost = 700 #Dollars
a_s_t = [(0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0),
         (1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1)] #Binary Paramters to specify if hour in Schedule

#SAA - Global Parameters
numscen = 1000 #N
numbatch = 10 #M
numscen_saa = 100
numeval = 500 #N_tilde
#numscen = Arrival_Rate_Data['Scenario '].max()
#numscen = 10
Cvar_alpha = 0.1
Cvar_lambda = 10

cand_xd = 0.0
cand_xe = 0.0
cand_yt = 0.0

arrival_rate = {}

# ALWAYS USE [k,t,i] instead of [t,k,i]
for k in range(numscen):
    for t in range(24):
        for i in range(6):
            #arrival_rate[t,k,i] = Arrival_Rate_Data.get_value(Arrival_Rate_Data[Arrival_Rate_Data['Code']==(str(k) + '_' + str(t) + '_' + str(i))].index,4, takeable = True)#Assumed for now, replace with data
             if i == 0:
                arrival_rate[k,t,i] = 1.2 #Historical Mean - Events arriving per hour
                arrival_rate[k,t,i] = np.random.poisson(lam = 1.2)
             elif i == 1:
                arrival_rate[k,t,i] = 1.3 #Historical Mean
                arrival_rate[k,t,i] = np.random.poisson(lam = 1.3)
             elif i == 2:
                arrival_rate[k,t,i] = 1.21 #Historical Mean
                arrival_rate[k,t,i] = np.random.poisson(lam = 1.21)
             elif i == 3:
                arrival_rate[k,t,i] = 1.3 #Historical Mean
                arrival_rate[k,t,i] = np.random.poisson(lam = 1.3)
             elif i == 4:
                arrival_rate[k,t,i] = 2 #Historical Mean
                arrival_rate[k,t,i] = np.random.poisson(lam = 2)
             else:
                arrival_rate[k,t,i] = 1.5 #Historical Mean
                arrival_rate[k,t,i] = np.random.poisson(lam = 1.5)
                                
service_rate = {}
for t in range(24):
    for i in range(6):
        if i == 0:
            service_rate[t,i] = random.expovariate(1/1.52)
            
        elif i == 1:
            service_rate[t,i] = random.expovariate(1/1.2)
        elif i == 2:
            service_rate[t,i] = random.expovariate(1/1.61)
        elif i ==3:
            service_rate[t,i] = random.expovariate(1/2.1)
        elif i == 4:
            service_rate[t,i] = random.expovariate(1/1.02)
        else: 
            service_rate[t,i] = random.expovariate(1/1.01)
CutViolationTolerance = 0.000001

In [3]:
#Find probability of event - to replace the 1/float(numscen) terms used below:
def calc_prob_scenario(events_under_scenario,mean_events, distribution):
    if distribution == 0: #Poisson
        #Reference: https://stattrek.com/probability-distributions/poisson.aspx
        return((np.exp(-mean_events)*(mean_events**events_under_scenario))/math.factorial(events_under_scenario))
    else:
        return(1 - np.exp((-1/mean_events)*events_under_scenario))

### Stochastic Programming Formulation - Extensive Formulation

In [4]:
def extensive_formulation(service_rate, arrival_rate, numscen):
    m_ef = Model('ExtensiveForm')
    #m_ef.params.logtoconsole = 0
    
    #First Stage Decision Variables: 
    #Number of Trouble Truck Crews working on Day Shift
    x_d = m_ef.addVar(vtype = 'INTEGER', lb = 0, obj = Trbl_Trck_Hr_Rate * Trbl_Shift_Duration)
    
    #Number of Trouble Truck Crews working on Night Shift
    x_e = m_ef.addVar(vtype = 'INTEGER', lb = 0, obj = Trbl_Trck_Hr_Rate * Trbl_Shift_Duration)
    
    #Number of Trouble Truck Crews working at time t
    y_t = {}
    for t in range(24):
        y_t[t] = m_ef.addVar(vtype = 'INTEGER', obj = 0)
    
    #Second Stage Decision Variables:
    #Number of events that will be met at time t
    v_itk = {}
    #Number of events that will be unment at time t
    w_itk = {}
    for k in range(numscen):
        for t in range(24):
            for i in range(6):
                v_itk[k,t,i] = m_ef.addVar(vtype = 'CONTINOUS', lb = 0, obj = 0)
                w_itk[k,t,i] = m_ef.addVar(vtype = 'CONTINOUS', lb = 0, obj = Penalty_Cost/float(numscen))
    m_ef.update()
    
    #First Stage Constraints: 
    m_ef.addConstrs(x_d * a_s_t[0][t] + x_e * a_s_t[1][t] >= y_t[t] for t in range(24))
    m_ef.addConstr(x_d + x_e <= 6) #Total Number of crews procured in a day cannot exceed 6
    
    m_ef.addConstr(2*x_d >= 3*x_e) # Day shift crews > Night Shift Crews - NEW CONSTRAINT - Can comment out
    
    #Second Stage Constraints: 
    for k in range(numscen):
        for t in range(24):
            for i in range(6):
                m_ef.addConstr((service_rate[t,i] * v_itk[k,t,i]) + w_itk[k,t,i] == arrival_rate[k,t,i])
                #if i == 0:
                    #m_ef.addConstr((service_rate[t,i] * v_itk[k,t,i]) >= 0.8 * arrival_rate[k,t,i])
                #if i == 0: # Category 1 events must be met 80% of the time
                    #m_ef.addConstr((service_rate[t,i] * v_itk[k,t,i]) + w_itk[k,t,i] >= 0.8 * arrival_rate[k,t,i])
                #else: 
                    #m_ef.addConstr((service_rate[t,i] * v_itk[k,t,i]) + w_itk[k,t,i] == arrival_rate[k,t,i])
    
    m_ef.addConstrs(sum(v_itk[k,t,i] for i in range(6)) <= y_t[t] for t in range(24) for k in range(numscen))

    m_ef.modelSense = GRB.MINIMIZE
    m_ef.update()
    start = time.time()
    m_ef.optimize()
    end = time.time()
    print("The time used for extensive formulation is: ", end-start)
    y_output = {}
    for t in range(24):
        y_output[t] = y_t[t].x
    return[m_ef.objval, x_d.x, x_e.x, y_output, end-start]

In [5]:
extensive_formulation(service_rate, arrival_rate, numscen)

Academic license - for non-commercial use only
Optimize a model with 168026 rows, 288026 columns and 456052 nonzeros
Variable types: 288000 continuous, 26 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-03, 8e+00]
  Objective range  [7e-01, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 143684.80000
Presolve removed 164020 rows and 273839 columns
Presolve time: 1.22s
Presolved: 4006 rows, 14187 columns, 18193 nonzeros
Variable types: 14035 continuous, 152 integer (126 binary)

Root relaxation: objective 8.480902e+04, 5908 iterations, 0.35 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 84809.0244    0   26 143684.800 84809.0244  41.0%     -    1s
H    0     0                    85758.738774 84809.0244  1.11%     -    1s
     0     0     cutoff    0      85758.7388 85758.7388  0.00% 

[85758.73877430527,
 4.0,
 2.0,
 {0: 2.0,
  1: 2.0,
  2: 2.0,
  3: 2.0,
  4: 2.0,
  5: 2.0,
  6: 2.0,
  7: 4.0,
  8: 4.0,
  9: 4.0,
  10: 4.0,
  11: 4.0,
  12: 4.0,
  13: 4.0,
  14: 4.0,
  15: 4.0,
  16: 4.0,
  17: 4.0,
  18: 4.0,
  19: 2.0,
  20: 2.0,
  21: 2.0,
  22: 2.0,
  23: 2.0},
 1.8885750770568848]

### Risk Measure - CVaR

In [6]:
def extensive_form_with_CVaR(service_rate, arrival_rate, numscen):
    C_ef = Model('ExtensiveForm with CVaR')
    #C_ef.params.logtoconsole = 0
    
    #First Stage Decision Variables: 
    #Number of Trouble Truck Crews working on Day Shift
    x_d_c = C_ef.addVar(vtype = 'INTEGER', lb = 1, obj = (1+Cvar_lambda)*(Trbl_Trck_Hr_Rate * Trbl_Shift_Duration))
    
    #Number of Trouble Truck Crews working on Night Shift
    x_e_c = C_ef.addVar(vtype = 'INTEGER', lb = 1, obj = (1+Cvar_lambda)*(Trbl_Trck_Hr_Rate * Trbl_Shift_Duration))
    
    #Number of Trouble Truck Crews working at time t
    y_t_c = {}
    for t in range(24):
        y_t_c[t] = C_ef.addVar(vtype = 'INTEGER')
    
    #CVaR Variables:
    r_c = C_ef.addVar(vtype = 'CONTINOUS', obj = Cvar_lambda)
    
    #Second Stage Decision Variables:
    #Number of events that will be met at time t
    v_itk_c = {}
    #Number of events that will be unment at time t
    w_itk_c = {}
    #Extra vars for CVaR
    u_k_c = {}
    for k in range(numscen):
        for t in range(24):
            for i in range(6):
                v_itk_c[k,t,i] = C_ef.addVar(vtype = 'CONTINOUS', lb = 0, obj = 0)
                w_itk_c[k,t,i] = C_ef.addVar(vtype = 'CONTINOUS', lb = 0,  obj = Penalty_Cost/float(numscen) )
        u_k_c[k] = C_ef.addVar(vtype = 'CONTINOUS', lb = 0, obj = (Cvar_lambda/(1-Cvar_alpha))*(1/float(numscen)))
        
    C_ef.update()
    
    #First Stage Constraints: 
    C_ef.addConstrs(x_d_c * a_s_t[0][t] + x_e_c * a_s_t[1][t] >= y_t_c[t] for t in range(24))
    C_ef.addConstr(x_d_c + x_e_c <= 6) #Total Number of crews procured in a day cannot exceed 6
    C_ef.addConstr(2 * x_d_c >= 3 * x_e_c) # Day shift crews > Night Shift Crews - Can comment out
    
    #Second Stage Constraints: 
    for k in range(numscen):
        for t in range(24):
            for i in range(6):
                C_ef.addConstr((service_rate[t,i] * v_itk_c[k,t,i]) + w_itk_c[k,t,i] == arrival_rate[k,t,i])
                #if i == 0:
                    #C_ef.addConstr((service_rate[t,i] * v_itk[k,t,i]) + w_itk[k,t,i] >= 0.8 * arrival_rate[k,t,i])
                #else: 
                    #C_ef.addConstr((service_rate[t,i] * v_itk[k,t,i]) + w_itk[k,t,i] == arrival_rate[k,t,i])
            
    C_ef.addConstrs(sum(v_itk_c[k,t,i] for i in range(6)) <= y_t_c[t] for t in range(24) for k in range(numscen))
    C_ef.addConstrs(u_k_c[k] >= sum((Penalty_Cost)*w_itk_c[k,t,i] for i in range(6) for t in range(24)) - r_c for k in range(numscen))

    C_ef.modelSense = GRB.MINIMIZE
    C_ef.update()
    
    start = time.time()
    C_ef.optimize()
    end = time.time()
    print("The time used for extensive formulation is: ", end-start)
    y_output_c = {}
    for t in range(24):
        y_output_c[t] = y_t_c[t].x
    
    return[C_ef.objval, x_d_c.x, x_e_c.x, y_output_c, end-start]

In [7]:
extensive_form_with_CVaR(service_rate, arrival_rate, numscen)

Optimize a model with 169026 rows, 289027 columns and 602052 nonzeros
Variable types: 289001 continuous, 26 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-03, 7e+02]
  Objective range  [1e-02, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 144253 rows and 193604 columns
Presolve time: 0.94s
Presolved: 24773 rows, 95423 columns, 214842 nonzeros
Variable types: 95363 continuous, 60 integer (34 binary)
Found heuristic solution: objective 958407.19896

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   32726    9.1790301e+05   1.939937e+04   0.000000e+00      5s
   37612    9.4345843e+05   0.000000e+00   0.000000e+00      6s

Root relaxation: objective 9.434584e+05, 37612 iterations, 4.27 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      958407.199 

[958407.1989628347,
 4.0,
 2.0,
 {0: 2.0,
  1: 2.0,
  2: 2.0,
  3: 2.0,
  4: 2.0,
  5: 2.0,
  6: 2.0,
  7: 4.0,
  8: 4.0,
  9: 4.0,
  10: 4.0,
  11: 4.0,
  12: 4.0,
  13: 4.0,
  14: 4.0,
  15: 4.0,
  16: 4.0,
  17: 4.0,
  18: 4.0,
  19: 2.0,
  20: 2.0,
  21: 2.0,
  22: 2.0,
  23: 2.0},
 7.242433071136475]

### Benders Decomposition - Single-Cut Benders

In [8]:
##### Build the single-cut master problem #####
mp_s = Model("MP_singlecut")
mp_s.Params.outputFlag = 0  # turn off output
# MP.Params.method = 1      # dual simplex
mp_s.params.logtoconsole = 0 # Turn off Gurobi log of progress

#First Stage Decision Variables: 
#Number of Trouble Truck Crews working on Day Shift
x_d_s = mp_s.addVar(vtype = 'INTEGER', lb = 1, obj = Trbl_Trck_Hr_Rate * Trbl_Shift_Duration)
#Number of Trouble Truck Crews working on Night Shift
x_e_s = mp_s.addVar(vtype = 'INTEGER', lb = 1, obj = Trbl_Trck_Hr_Rate * Trbl_Shift_Duration)
# Theta
theta_s = mp_s.addVar(vtype = 'CONTINUOUS', lb = 0, obj = 1)
#Number of Trouble Truck Crews working at time t
y_t_s = {}
for t in range(24):
    y_t_s[t] = mp_s.addVar(vtype = 'INTEGER', lb = 0, obj = 0)

#First Stage Constraints: 
mp_s.addConstrs(x_d_s * a_s_t[0][t] + x_e_s * a_s_t[1][t] >= y_t_s[t] for t in range(24))
mp_s.addConstr(x_d_s + x_e_s <= 6) #Total Number of crews procured in a day cannot exceed 6
mp_s.addConstr(2 * x_d_s >= 3 * x_e_s) # Day shift crews > Night Shift Crews - Can comment out

# Note: Default variable bounds are LB = 0 and UB = infinity
mp_s.modelSense = GRB.MINIMIZE

In [9]:
##### Build the subproblem(s) #####
# Build Primal SP
SP = Model("SP")

#SECOND STAGE RANDOM VARIABLES: PRIMAL SP
#Number of events that will be met at time t
v_itk_sp = {}
#Number of events that will be unment at time t
w_itk_sp = {}
for t in range(24):
    for i in range(6):
        v_itk_sp[t,i] = SP.addVar(vtype = 'CONTINOUS', lb = 0, obj = 0)
        w_itk_sp[t,i] = SP.addVar(vtype = 'CONTINOUS', lb = 0, obj = Penalty_Cost)

#CONSTRAINT ARRAYS TO STORE CONSTRAINTS
IncomingArrivals = {}
CrewAllocation = {}
            
#Second Stage Constraints: 
for t in range(24):
    for i in range(6):
        IncomingArrivals[t,i] = SP.addConstr(((service_rate[t,i] * v_itk_sp[t,i]) + w_itk_sp[t,i] == 0), "Arrival"+str(k)+str(t)+str(i))
        # 0 -> arrival_rate[k,t,i]

for t in range(24):
    CrewAllocation[t] = SP.addConstr(sum(v_itk_sp[t,i] for i in range(6)) <= 0 , "Allocation"+str(k)+str(t))
# 0 -> y_t[t]

SP.modelSense = GRB.MINIMIZE
SP.Params.outputFlag = 0

In [10]:
##### Modify SP to Get Benders Cut #####
def ModifyAndSolveSP_SingleCut(k,y_sol):
    S_ind = k
    # Modify constraint rhs
    #Second Stage Constraints: 
    for t in range(24):
        for i in range(6):
            IncomingArrivals[t,i].rhs = arrival_rate[k,t,i]
    
    for t in range(24):
        CrewAllocation[t].rhs = y_sol[t]
    
    SP.params.logtoconsole=0 
    SP.update()

    # Solve and get the DUAL solution
    SP.optimize()
    
    pi_1_sol = {}
    pi_2_sol = {}
    for t in range(24):
        pi_2_sol[t] = CrewAllocation[t].Pi
        for i in range(6):
            pi_1_sol[t,i] = IncomingArrivals[t,i].Pi

    SPobj = SP.objVal

    # Check whether a violated Benders cut is found
    CutFound_ = False
    #if(eta_sol_m[S_ind] < SPobj - CutViolationTolerance): # Found Benders cut is violated at the current master solution
        #CutFound_ = True
    return SPobj, CutFound_, pi_1_sol, pi_2_sol

In [11]:
##### Single-Cut Benders Loop #####
def SingleCutBenders():
    CutFound = True
    NoIters = 0
    NoCut = 0
    #OptFound = False
    BestLB = -GRB.INFINITY
    BestUB = GRB.INFINITY
    xsol = [None] * 2  #two vars: only x_d and x_e
    y_sol = [None] *24
    LB = 0
    UB = 0
    start = time.time()
    while(CutFound and NoIters < 20):
        NoIters += 1
        CutFound = False
        UB = 0

        # Solve MP
        mp_s.params.logtoconsole=0 
        mp_s.update()
        mp_s.optimize()

        # Get MP solution
        MPobj_s = mp_s.objVal
        LB = MPobj_s
        if LB > BestLB:
            BestLB = LB

        xsol[0] = x_d_s.x
        xsol[1] = x_e_s.x
        for t in range(24):
            y_sol[t] = y_t_s[t].x
        #print(xsol[0])
        #print(xsol[1])
        UB += Trbl_Trck_Hr_Rate * Trbl_Shift_Duration * (xsol[0]+xsol[1])
        #print(UB)
        thetasol = theta_s.x
        SPobj_total_s = 0
        expr_single = {}

        for k in range(numscen):
            Qvalue_s, CutFound_s, pi_1_sol_s, pi_2_sol_s = ModifyAndSolveSP_SingleCut(k,y_sol)
            UB += 1/float(numscen) * Qvalue_s
            expr_s = {}
            SPobj_total_s += 1/float(numscen) * Qvalue_s # For Single Cut Benders
            expr_s = LinExpr(- quicksum(arrival_rate[k,t,i] * pi_1_sol_s[t,i] for t in range(24) for i in range (6))
                             - quicksum(y_t_s[t] * pi_2_sol_s[t] for t in range(24)))
            expr_single += expr_s

        if(thetasol < SPobj_total_s - CutViolationTolerance):
            CutFound = True
            mp_s.addConstr(theta_s + expr_single/numscen >= 0)
            NoCut += 1

        if(UB < BestUB):
            BestUB = UB
        print("Iteration: ", NoIters)
        print("UB: " + str(UB))
        print("LB: " + str(LB))

    end = time.time()
    print()
    if BestUB - BestLB <= 0.000001:
        print("The optimal solution has been found with an objective value: ", BestUB)
    else:
        print('BestLB: %g' % BestLB)
        print('BestUB: %g' % BestUB)

    print("Time used is: ", end - start)
    print("Total number of cuts added is: ", NoCut)
    print("Number of Iterations is : " + str(NoIters))
    
    return[BestUB, xsol[0], xsol[1], y_sol, end-start]

In [12]:
SingleCutBenders()

Iteration:  1
UB: 152684.79999999993
LB: 9000.0
Iteration:  2
UB: 93306.39676872913
LB: 15000.0
Iteration:  3
UB: 85758.7387743006
LB: 83026.34612165255
Iteration:  4
UB: 85758.7387743006
LB: 85758.73877430076

The optimal solution has been found with an objective value:  85758.7387743006
Time used is:  3.7031190395355225
Total number of cuts added is:  3
Number of Iterations is : 4


[85758.7387743006,
 4.0,
 2.0,
 [2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0],
 3.7031190395355225]

### Benders Decomposition - Multi-Cut Benders

In [13]:
##### Build the multi-cut master problem #####
mp_m = Model("MP_multicut")
mp_m.Params.outputFlag = 0  # turn off output
# MP.Params.method = 1      # dual simplex
mp_m.params.logtoconsole = 0 # Turn off Gurobi log of progress

#First Stage Decision Variables: 
#Number of Trouble Truck Crews working on Day Shift
x_d_m = mp_m.addVar(vtype = 'INTEGER', lb = 1, obj = Trbl_Trck_Hr_Rate * Trbl_Shift_Duration)
#Number of Trouble Truck Crews working on Night Shift
x_e_m = mp_m.addVar(vtype = 'INTEGER', lb = 1, obj = Trbl_Trck_Hr_Rate * Trbl_Shift_Duration)
# eta
eta_m = {}
for k in range(numscen):
    eta_m[k] = mp_m.addVar(vtype = 'CONTINUOUS', lb = 0, obj = 1/float(numscen))
#Number of Trouble Truck Crews working at time t
y_t_m = {}
for t in range(24):
    y_t_m[t] = mp_m.addVar(vtype = 'INTEGER', lb = 0, obj = 0)

#First Stage Constraints: 
mp_m.addConstrs(x_d_m * a_s_t[0][t] + x_e_m * a_s_t[1][t] >= y_t_m[t] for t in range(24))
mp_m.addConstr(x_d_m + x_e_m <= 6) #Total Number of crews procured in a day cannot exceed 6
mp_m.addConstr(2 * x_d_m >= 3 * x_e_m) # Day shift crews > Night Shift Crews - Can comment out

# Note: Default variable bounds are LB = 0 and UB = infinity
mp_m.modelSense = GRB.MINIMIZE

In [14]:
##### Modify SP to Get Benders Cut #####
def ModifyAndSolveSP_MultiCut(k,y_sol,eta_sol_m):
    S_ind = k
    # Modify constraint rhs
    #Second Stage Constraints: 
    for t in range(24):
        for i in range(6):
            IncomingArrivals[t,i].rhs = arrival_rate[k,t,i]
    
    for t in range(24):
        CrewAllocation[t].rhs = y_sol[t]
    
    SP.params.logtoconsole=0 
    SP.update()

    # Solve and get the DUAL solution
    SP.optimize()
    
    pi_1_sol = {}
    pi_2_sol = {}
    for t in range(24):
        pi_2_sol[t] = CrewAllocation[t].Pi
        for i in range(6):
            pi_1_sol[t,i] = IncomingArrivals[t,i].Pi

    SPobj = SP.objVal

    # Check whether a violated Benders cut is found
    CutFound_ = False
    if(eta_sol_m[S_ind] < SPobj - CutViolationTolerance): # Found Benders cut is violated at the current master solution
        CutFound_ = True
    return SPobj, CutFound_, pi_1_sol, pi_2_sol

In [15]:
def regular_multicut_benders():
    ##### Multi-Cut Benders Loop #####
    CutFound = True
    NoIters = 0
    NoCut = 0
    #OptFound = False
    BestLB = -GRB.INFINITY
    BestUB = GRB.INFINITY
    xsol = [None] * 2  #two vars: only x_d and x_e
    y_sol = [None] *24
    LB = 0
    UB = 0
    start = time.time()
    while(CutFound and NoIters < 20):
        NoIters += 1
        CutFound = False
        UB = 0

        # Solve MP
        mp_m.params.logtoconsole=0 
        mp_m.update()
        mp_m.optimize()

        # Get MP solution
        MPobj_m = mp_m.objVal
        LB = MPobj_m
        if LB > BestLB:
            BestLB = LB

        xsol[0] = x_d_m.x
        xsol[1] = x_e_m.x
        for t in range(24):
            y_sol[t] = y_t_m[t].x
        UB += Trbl_Trck_Hr_Rate * Trbl_Shift_Duration * (xsol[0]+xsol[1])
        eta_sol_m = {}
        for k in range(numscen):
            eta_sol_m[k] = eta_m[k].x

        for k in range(numscen):
            Qvalue_m, CutFound_m, pi_1_sol_m, pi_2_sol_m = ModifyAndSolveSP_MultiCut(k,y_sol,eta_sol_m)
            UB += 1/float(numscen) * Qvalue_m
            if(CutFound_m):
                CutFound = True
                expr = LinExpr(eta_m[k] - quicksum(arrival_rate[k,t,i] * pi_1_sol_m[t,i] for t in range(24) for i in range (6))
                               - quicksum(y_t_m[t] * pi_2_sol_m[t] for t in range(24)))
                mp_m.addConstr(expr >= 0)
                NoCut += 1

        if(UB < BestUB):
            BestUB = UB
        print("Iteration: ", NoIters)
        print("UB: " + str(UB))
        print("LB: " + str(LB))

    end = time.time()
    print()
    if BestUB - BestLB <= 0.000001:
        print("The optimal solution has been found with an objective value: ", BestUB)
    else:
        print('BestLB: %g' % BestLB)
        print('BestUB: %g' % BestUB)

    print("Time used is: ", end - start)
    print("Total number of cuts added is: ", NoCut)
    print("Number of Iterations is : " + str(NoIters))
    return[BestUB, xsol[0], xsol[1], y_sol, end-start]

In [16]:
regular_multicut_benders()

Iteration:  1
UB: 152684.79999999993
LB: 9000.0
Iteration:  2
UB: 93306.39676872913
LB: 15191.154041923435
Iteration:  3
UB: 85758.7387743006
LB: 83026.34612165263
Iteration:  4
UB: 85758.7387743006
LB: 85758.73877430073

The optimal solution has been found with an objective value:  85758.7387743006
Time used is:  3.51971435546875
Total number of cuts added is:  3000
Number of Iterations is : 4


[85758.7387743006,
 4.0,
 2.0,
 [2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0],
 3.51971435546875]

### Level Method - Single-Cut

In [17]:
##### Build the single-cut master problem #####
mp_s = Model("MP_singlecut")
mp_s.Params.outputFlag = 0  # turn off output
# MP.Params.method = 1      # dual simplex
mp_s.params.logtoconsole = 0 # Turn off Gurobi log of progress

#First Stage Decision Variables: 
#Number of Trouble Truck Crews working on Day Shift
x_d_s = mp_s.addVar(vtype = 'INTEGER', lb = 1, obj = Trbl_Trck_Hr_Rate * Trbl_Shift_Duration)
#Number of Trouble Truck Crews working on Night Shift
x_e_s = mp_s.addVar(vtype = 'INTEGER', lb = 1, obj = Trbl_Trck_Hr_Rate * Trbl_Shift_Duration)
# Theta
theta_s = mp_s.addVar(vtype = 'CONTINUOUS', lb = 0, obj = 1)
#Number of Trouble Truck Crews working at time t
y_t_s = {}
for t in range(24):
    y_t_s[t] = mp_s.addVar(vtype = 'INTEGER', lb = 0, obj = 0)

#First Stage Constraints: 
mp_s.addConstrs(x_d_s * a_s_t[0][t] + x_e_s * a_s_t[1][t] >= y_t_s[t] for t in range(24))
mp_s.addConstr(x_d_s + x_e_s <= 6) #Total Number of crews procured in a day cannot exceed 6
mp_s.addConstr(2 * x_d_s >= 3 * x_e_s) # Day shift crews > Night Shift Crews - Can comment out

# Note: Default variable bounds are LB = 0 and UB = infinity
mp_s.modelSense = GRB.MINIMIZE

In [18]:
##### Modify SP to Get Benders Cut #####
def LevelSP_SingleCut(k,y_t_t,y_sol):
    S_ind = k
    # Modify constraint rhs
    #Second Stage Constraints: 
    for t in range(24):
        y_sol[t] = y_t_t[t]
    
    for t in range(24):
        for i in range(6):
            IncomingArrivals[t,i].rhs = arrival_rate[k,t,i]
    
    for t in range(24):
        CrewAllocation[t].rhs = y_sol[t]
    
    SP.params.logtoconsole=0 
    SP.update()

    # Solve and get the DUAL solution
    SP.optimize()
    
    pi_1_sol = {}
    pi_2_sol = {}
    for t in range(24):
        pi_2_sol[t] = CrewAllocation[t].Pi
        for i in range(6):
            pi_1_sol[t,i] = IncomingArrivals[t,i].Pi

    SPobj = SP.objVal

    # Check whether a violated Benders cut is found
    CutFound_ = False

    return SPobj, CutFound_, pi_1_sol, pi_2_sol

In [19]:
def Level_SingleCut():
    lambda_ = 0.29 # 0.25 is better ! maybe 0.22 is better even
    termination_tolerance = 0.000001
    termination_flag = 0
    UB_flag = 0
    LB_flag = 0
    CutFound = False
    NoIters = 0
    NoCut = 0
    #OptFound = False
    BestLB = -GRB.INFINITY
    BestUB = GRB.INFINITY
    #xsol = [0] * 2  #two vars: only x_d and x_e
    y_sol = {}
    for t in range(24):
        y_sol[t] = 0
    theta_sol_m = 0
    x_d_t = 0
    x_e_t = 0
    y_t_t = {}
    for t in range(24):
        y_t_t[t] = 0

    start = time.time()
    while(NoIters < 10):
        LB = 0
        UB = 0
        NoIters += 1
        CutFound = False    

        ## Solve subproblems and update m^t+1(x)
        if NoIters > 1:
            theta_sol_m = theta_mt_x
        SPobj_total_s = 0
        expr_single = {}
        
        UB += Trbl_Trck_Hr_Rate * Trbl_Shift_Duration * (x_d_t + x_e_t)
        #print(x_d_t,x_e_t)
        for k in range(numscen):
            Qvalue_s, CutFound_s, pi_1_sol_s, pi_2_sol_s = LevelSP_SingleCut(k,y_t_t,y_sol)
            UB += 1/float(numscen) * Qvalue_s
            expr_s = {}
            SPobj_total_s += 1/float(numscen) * Qvalue_s # For Single Cut Benders
            expr_s = LinExpr(- quicksum(arrival_rate[k,t,i] * pi_1_sol_s[t,i] for t in range(24) for i in range (6))
                             - quicksum(y_t_s[t] * pi_2_sol_s[t] for t in range(24)))
            expr_single += expr_s

        if(theta_sol_m < SPobj_total_s - CutViolationTolerance):
            CutFound = True
            mp_s.addConstr(theta_s + expr_single/numscen >= 0)
            NoCut += 1
            
        if UB == BestUB:
            UB_flag = 1
        if UB < BestUB:
            BestUB = UB  # zt^bar is our BestUB

        ## Solve MP to get zt_bar
        mp_s.update()
        mp_s.optimize()
        zt_bar = 0
        zt_bar = mp_s.objVal
        LB = zt_bar
        if LB == BestLB:
            LB_flag = 1
        if LB > BestLB:
            BestLB = LB
        theta_mt = mp_s.getVars()[2]
        theta_mt_x = theta_mt.x
        print("Iteration: ", NoIters)
        print("UB: " + str(UB))
        print("LB: " + str(LB))

        ## Optimality check
        if BestUB - BestLB <= termination_tolerance or CutFound == False: #  or (UB_flag == 1 and LB_flag == 1)
            end_opt = time.time()
            print("The optimal solution has been found with an objective value: ", BestUB)
            print("Time used is: ", end_opt - start)
            print("Total number of cuts added is: ", NoCut)
            print("Number of Iterations is : " + str(NoIters))
            termination_flag = 1
            break

        ## Compute l_t
        l_t = zt_bar + lambda_*(BestUB - zt_bar)

        ## Compute x_tp1
        const_lt = (mp_s.addConstr(theta_s+
                                      Trbl_Trck_Hr_Rate*Trbl_Shift_Duration*x_d_s+
                                      Trbl_Trck_Hr_Rate * Trbl_Shift_Duration*x_e_s-l_t<=0))
        obj_1 = (x_d_s-x_d_t)*(x_d_s-x_d_t)
        obj_2 = (x_e_s-x_e_t)*(x_e_s-x_e_t)
        obj_3 = quicksum((y_t_s[t] - y_t_t[t])*(y_t_s[t] - y_t_t[t]) for t in range(24))
        mp_s.setObjective(obj_1+obj_2+obj_3)
        mp_s.update()
        mp_s.optimize()

        ## Update x_t to x_tp1
        x_d_tp1 = mp_s.getVars()[0]
        x_e_tp1 = mp_s.getVars()[1]
        y_t_tp1 = mp_s.getVars()[3:27]
        x_d_t = x_d_tp1.x
        x_e_t = x_e_tp1.x
        for t in range(24):
            y_t_t[t] = y_t_tp1[t].x

        ## Update QP 
        obj_4 = Trbl_Trck_Hr_Rate*Trbl_Shift_Duration*(x_d_s+x_e_s) + theta_s
        mp_s.remove(const_lt)
        mp_s.setObjective(obj_4)
        #mp_m.update()

    if termination_flag != 1:
        end_opt = time.time()
        print('BestLB: %g' % BestLB)
        print('BestUB: %g' % BestUB)
        print("Time used is: ", end_opt - start)
        print("Total number of cuts added is: ", NoCut)
        print("Number of Iterations is : " + str(NoIters))
        
    return[BestUB, x_d_t, x_e_t, y_t_t, end_opt-start]

In [20]:
Level_SingleCut()

Iteration:  1
UB: 143684.7999999999
LB: 15000.0
Iteration:  2
UB: 107947.16977858635
LB: 72436.37117440975
Iteration:  3
UB: 88922.19805745155
LB: 84268.65745176825
Iteration:  4
UB: 86629.46704021603
LB: 85646.57969347895
Iteration:  5
UB: 85946.1826834419
LB: 85729.65468621612
Iteration:  6
UB: 85758.7387743006
LB: 85758.73877430076
The optimal solution has been found with an objective value:  85758.7387743006
Time used is:  5.463807582855225
Total number of cuts added is:  6
Number of Iterations is : 6


[85758.7387743006,
 4.0,
 2.0,
 {0: 2.0,
  1: 2.0,
  2: 2.0,
  3: 2.0,
  4: 2.0,
  5: 2.0,
  6: 2.0,
  7: 4.0,
  8: 4.0,
  9: 4.0,
  10: 4.0,
  11: 4.0,
  12: 4.0,
  13: 4.0,
  14: 4.0,
  15: 4.0,
  16: 4.0,
  17: 4.0,
  18: 4.0,
  19: 2.0,
  20: 2.0,
  21: 2.0,
  22: 2.0,
  23: 2.0},
 5.463807582855225]

### Level Method - Multi-Cut - Sometimes Cannot Get Correct Optimal Solutions

In [21]:
##### Build the multi-cut master problem ##### -- Not in report!
mp_m = Model("MP_multicut")
mp_m.Params.outputFlag = 0  # turn off output
# MP.Params.method = 1      # dual simplex
mp_m.params.logtoconsole = 0 # Turn off Gurobi log of progress

#First Stage Decision Variables: 
#Number of Trouble Truck Crews working on Day Shift
x_d_m = mp_m.addVar(vtype = 'INTEGER', lb = 1, obj = Trbl_Trck_Hr_Rate * Trbl_Shift_Duration)
#Number of Trouble Truck Crews working on Night Shift
x_e_m = mp_m.addVar(vtype = 'INTEGER', lb = 1, obj = Trbl_Trck_Hr_Rate * Trbl_Shift_Duration)
# eta
eta_m = {}
for k in range(numscen):
    eta_m[k] = mp_m.addVar(vtype = 'CONTINUOUS', lb = 0, obj = 1/float(numscen))
#Number of Trouble Truck Crews working at time t
y_t_m = {}
for t in range(24):
    y_t_m[t] = mp_m.addVar(vtype = 'INTEGER', lb = 0, obj = 0)

#First Stage Constraints: 
mp_m.addConstrs(x_d_m * a_s_t[0][t] + x_e_m * a_s_t[1][t] >= y_t_m[t] for t in range(24))
mp_m.addConstr(x_d_m + x_e_m <= 6) #Total Number of crews procured in a day cannot exceed 6
mp_m.addConstr(2 * x_d_m >= 3 * x_e_m) # Day shift crews > Night Shift Crews - Can comment out

# Note: Default variable bounds are LB = 0 and UB = infinity
mp_m.modelSense = GRB.MINIMIZE

In [22]:
##### Modify SP to Get Benders Cut #####
def LevelSP_MultiCut(k,y_t_t,eta_sol_m,y_sol):
    S_ind = k
    # Modify constraint rhs
    #Second Stage Constraints: 
    for t in range(24):
        y_sol[t] = y_t_t[t]
    
    for t in range(24):
        for i in range(6):
            IncomingArrivals[t,i].rhs = arrival_rate[k,t,i]
    
    for t in range(24):
        CrewAllocation[t].rhs = y_sol[t]
    
    SP.params.logtoconsole=0 
    SP.update()

    # Solve and get the DUAL solution
    SP.optimize()
    
    pi_1_sol = {}
    pi_2_sol = {}
    for t in range(24):
        pi_2_sol[t] = CrewAllocation[t].Pi
        for i in range(6):
            pi_1_sol[t,i] = IncomingArrivals[t,i].Pi

    SPobj = SP.objVal

    # Check whether a violated Benders cut is found
    CutFound_ = False
    if(eta_sol_m[S_ind] < SPobj - CutViolationTolerance): # Found Benders cut is violated at the current master solution
        CutFound_ = True
    return SPobj, CutFound_, pi_1_sol, pi_2_sol

In [23]:
def Level_MultiCut():
    lambda_ = 0.29
    termination_tolerance = 0.000001
    termination_flag = 0
    UB_flag = 0
    LB_flag = 0
    CutFound = False
    NoIters = 0
    NoCut = 0
    #OptFound = False
    BestLB = -GRB.INFINITY
    BestUB = GRB.INFINITY
    #xsol = [0] * 2  #two vars: only x_d and x_e
    y_sol = {}
    for t in range(24):
        y_sol[t] = 0
    eta_sol_m = {}
    for k in range(numscen):
        eta_sol_m[k] = 0
    x_d_t = 0
    x_e_t = 0
    y_t_t = {}
    for t in range(24):
        y_t_t[t] = 0

    start = time.time()
    while(NoIters < 10):
        LB = 0
        UB = 0
        NoIters += 1
        CutFound = False    

        ## Solve subproblems and update m^t+1(x)
        if NoIters > 1:
            for k in range(numscen):
                eta_sol_m[k] = eta_mt_x

        UB += Trbl_Trck_Hr_Rate * Trbl_Shift_Duration * (x_d_t + x_e_t)
        #print(x_d_t,x_e_t)
        for k in range(numscen):
            Qvalue_m, CutFound_m, pi_1_sol_m, pi_2_sol_m = LevelSP_MultiCut(k,y_t_t,eta_sol_m,y_sol)
            UB += 1/float(numscen) * Qvalue_m
            expr = {}
            if(CutFound_m):
                CutFound = True
                expr = LinExpr(eta_m[k] - quicksum(arrival_rate[k,t,i] * pi_1_sol_m[t,i] for t in range(24) for i in range (6))
                                - quicksum(y_t_m[t] * pi_2_sol_m[t] for t in range(24)))
                mp_m.addConstr(expr >= 0)
                NoCut += 1
        if UB == BestUB:
            UB_flag = 1
        if UB < BestUB:
            BestUB = UB  # zt^bar is our BestUB

        ## Solve MP to get zt_bar
        #MP.params.logtoconsole=0 
        mp_m.update()
        mp_m.optimize()
        zt_bar = 0
        zt_bar = mp_m.objVal
        LB = zt_bar
        if LB == BestLB:
            LB_flag = 1
        if LB > BestLB:
            BestLB = LB
        for k in range(numscen):
            eta_mt = mp_m.getVars()[1+k]
            eta_mt_x = eta_mt.x
        print("Iteration: ", NoIters)
        print("UB: " + str(UB))
        print("LB: " + str(LB))

        ## Optimality check
        if BestUB - BestLB <= termination_tolerance or CutFound == False or (UB_flag == 1 and LB_flag == 1): # or (UB_flag == 1 and LB_flag == 1)
            end = time.time()
            print("The optimal solution has been found with an objective value: ", BestUB)
            print("Time used is: ", end - start)
            print("Total number of cuts added is: ", NoCut)
            print("Number of Iterations is : " + str(NoIters))
            termination_flag = 1
            break

        ## Compute l_t
        l_t = zt_bar + lambda_*(BestUB - zt_bar)

        ## Compute x_tp1
        const_lt = (mp_m.addConstr(quicksum(eta_m[k]*1/float(numscen) for k in range(numscen))+
                                      Trbl_Trck_Hr_Rate*Trbl_Shift_Duration*x_d_m+
                                      Trbl_Trck_Hr_Rate * Trbl_Shift_Duration*x_e_m-l_t<=0))
        obj_1 = (x_d_m-x_d_t)*(x_d_m-x_d_t)
        obj_2 = (x_e_m-x_e_t)*(x_e_m-x_e_t)
        obj_3 = quicksum((y_t_m[t] - y_t_t[t])*(y_t_m[t] - y_t_t[t]) for t in range(24))
        mp_m.setObjective(obj_1+obj_2+obj_3)
        mp_m.update()
        mp_m.optimize()

        ## Update x_t to x_tp1
        x_d_tp1 = mp_m.getVars()[0]
        x_e_tp1 = mp_m.getVars()[1]
        y_t_tp1 = mp_m.getVars()[numscen+2:numscen+26]
        x_d_t = x_d_tp1.x
        x_e_t = x_e_tp1.x
        for t in range(24):
            y_t_t[t] = y_t_tp1[t].x
            # x_tp1_x[AllArcs[i]]

        ## Update QP 
        obj_4 = Trbl_Trck_Hr_Rate*Trbl_Shift_Duration*(x_d_m+x_e_m)
        obj_5 = quicksum(eta_m[k]/float(numscen) for k in range(numscen))
        mp_m.remove(const_lt)
        mp_m.setObjective(obj_4+obj_5)
        #mp_m.update()

    if termination_flag != 1:
        end = time.time()
        print('BestLB: %g' % BestLB)
        print('BestUB: %g' % BestUB)
        print("Time used is: ", end - start)
        print("Total number of cuts added is: ", NoCut)
        print("Number of Iterations is : " + str(NoIters))
        
    return[BestUB, x_d_t, x_e_t, y_t_t, end-start]

In [24]:
Level_MultiCut()

Iteration:  1
UB: 143684.7999999999
LB: 15191.154041923435
Iteration:  2
UB: 109228.16724569966
LB: 71268.73645840428
Iteration:  3
UB: 89124.55121331089
LB: 82067.91486089588
Iteration:  4
UB: 86753.72348462297
LB: 82500.60904819654
Iteration:  5
UB: 86327.40411074262
LB: 82507.38626448534
Iteration:  6
UB: 86139.96020160151
LB: 82514.2475474053
Iteration:  7
UB: 86139.96020160151
LB: 82514.2475474053
The optimal solution has been found with an objective value:  86139.96020160151
Time used is:  9.812272310256958
Total number of cuts added is:  3528
Number of Iterations is : 7


[86139.96020160151,
 4.0,
 2.0,
 {0: 2.0,
  1: 2.0,
  2: 2.0,
  3: 2.0,
  4: 2.0,
  5: 2.0,
  6: 2.0,
  7: 4.0,
  8: 4.0,
  9: 4.0,
  10: 4.0,
  11: 4.0,
  12: 4.0,
  13: 4.0,
  14: 4.0,
  15: 3.0,
  16: 4.0,
  17: 4.0,
  18: 4.0,
  19: 2.0,
  20: 2.0,
  21: 2.0,
  22: 2.0,
  23: 2.0},
 9.812272310256958]

### Sample Average Approximation

In [25]:
def extensive_formulation_SAA(arrival_rate_saa, service_rate_saa, numscen_saa):
    m_SAA_ef = Model('ExtensiveForm_SAA')
    m_SAA_ef.params.logtoconsole = 0
    
    #First Stage Decision Variables: 
    #Number of Trouble Truck Crews working on Day Shift
    x_d_saa = m_SAA_ef.addVar(vtype = 'INTEGER', lb = 0, obj = Trbl_Trck_Hr_Rate * Trbl_Shift_Duration)
    
    #Number of Trouble Truck Crews working on Night Shift
    x_e_saa = m_SAA_ef.addVar(vtype = 'INTEGER', lb = 0, obj = Trbl_Trck_Hr_Rate * Trbl_Shift_Duration)
    
    #Number of Trouble Truck Crews working at time t
    y_t_saa = {}
    for t in range(24):
        y_t_saa[t] = m_SAA_ef.addVar(vtype = 'INTEGER', obj = 0)
    m_SAA_ef.update()
    #Second Stage Decision Variables:
    #Number of events that will be met at time t
    v_itk_saa = {}
    #Number of events that will be unment at time t
    w_itk_saa = {}
    for k in range(numscen_saa):
        for t in range(24):
            for i in range(6):
                v_itk_saa[k,t,i] = m_SAA_ef.addVar(vtype = 'CONTINOUS', lb = 0, obj = 0)
                w_itk_saa[k,t,i] = m_SAA_ef.addVar(vtype = 'CONTINOUS', lb = 0,  obj = Penalty_Cost/float(numscen_saa))
    m_SAA_ef.update()
    
    #First Stage Constraints: 
    m_SAA_ef.addConstrs(x_d_saa * a_s_t[0][t] + x_e_saa*a_s_t[1][t] >= y_t_saa[t] for t in range(24))
    m_SAA_ef.addConstr(x_d_saa + x_e_saa <= 6) #Total Number of crews procured in a day cannot exceed 6
    m_SAA_ef.addConstr(2*x_d_saa >= 3*x_e_saa) # Day shift crews > Night Shift Crews - Can comment out
    #Second Stage Constraints: 
    for k in range(numscen_saa):
        for t in range(24):
            for i in range(6):
                m_SAA_ef.addConstr((service_rate_saa[t,i] * v_itk_saa[k,t,i]) + w_itk_saa[k,t,i] == arrival_rate_saa[k,t,i])
                #if i == 0: # Category 1 events must be met 80% of the time
                #    m_ef.addConstr((service_rate[t,i] * v_itk[t,k,i]) + w_itk[t,k,i] >= 0.8 * arrival_rate[t,k,i])
                #else: 
                #    m_ef.addConstr((service_rate[t,i] * v_itk[t,k,i]) + w_itk[t,k,i] == arrival_rate[t,k,i])
    
    m_SAA_ef.addConstrs(sum(v_itk_saa[k,t,i] for i in range(6)) <= y_t_saa[t] for t in range(24) for k in range(numscen_saa))

    m_SAA_ef.modelSense = GRB.MINIMIZE
    m_SAA_ef.update()
    m_SAA_ef.optimize()
    
    y_output_saa = {}
    for t in range(24):
        y_output_saa[t] = y_t_saa[t].x
    return[m_SAA_ef.objval, x_d_saa.x, x_e_saa.x, y_output_saa]

In [26]:
def SAA_solveSSP(x_d_val, x_e_val, y_t_val, service_rate_saa, arrival_rate_saa):
    SP_SAA = Model('Second Stage - SAA')
    SP_SAA.params.logtoconsole = 0
    v_it = {}
    w_it = {}
    for t in range(24):
        for i in range(6):
            v_it[t,i] = SP_SAA.addVar(vtype = 'CONTINOUS', lb = 0, obj = 0)
            w_it[t,i] = SP_SAA.addVar(vtype = 'CONTINOUS', lb = 0,  obj = Penalty_Cost)

    #Second Stage Constraints: 
    for t in range(24):
        for i in range(6):
            SP_SAA.addConstr((service_rate_saa[t,i] * v_it[t,i]) + w_it[t,i] == arrival_rate_saa[t,i])
            #if i == 0:
            #    SP_SAA.addConstr((service_rate[t,i] * v_it[t,i]) + w_it[t,i] == 0.8*arrival_rate[t,i])
            #else:
            #    SP_SAA.addConstr((service_rate[t,i] * v_it[t,i]) + w_it[t,i] == arrival_rate[t,i])
    SP_SAA.addConstrs(sum(v_it[t,i] for i in range(6)) <= y_t_val[t] for t in range(24))

    SP_SAA.modelSense = GRB.MINIMIZE
    SP_SAA.update()
    SP_SAA.optimize()
    
    SSP_sol = SP_SAA.objval
    return Trbl_Trck_Hr_Rate * Trbl_Shift_Duration*(x_d_val + x_e_val) + SSP_sol

In [27]:
#Main Procedure - In Progress
numbatch = 10 #M
numeval = 500  #N_tilde
numscen_saa = 100
objvals = np.zeros(numbatch)
soln_x_d = {}
soln_x_e = {}
soln_yt = {}
arrival_rate_SAA={}
service_rate_SAA = {}
### First separately estimate lower and upper bounds ####
# for LB:
rand_idx_lb = 0
for b in range(numbatch): # M
    for k in range(numscen_saa):
        rand_idx_lb = random.randint(0,numscen-1)
        for t in range(24):
            for i in range(6):
                arrival_rate_SAA[k,t,i] =  arrival_rate[rand_idx_lb,t,i]
                service_rate_SAA[t,i] = service_rate[t,i]
#                 if i == 0:
#                     arrival_rate_SAA[k,t,i] = np.random.poisson(lam = 1.2)
#                     service_rate_SAA[t,i] = np.random.exponential(scale = 1.52)
#                 elif i == 1:
#                     arrival_rate_SAA[k,t,i] = np.random.poisson(lam = 1.3)
#                     service_rate_SAA[t,i] = np.random.exponential(scale = 1.2)
#                 elif i == 2:
#                     arrival_rate_SAA[k,t,i] = np.random.poisson(lam = 1.2)
#                     service_rate_SAA[t,i] = np.random.exponential(scale = 1.61)
#                 elif i == 3:
#                     arrival_rate_SAA[k,t,i] = np.random.poisson(lam = 1.3)
#                     service_rate_SAA[t,i] = np.random.exponential(scale = 2.1)
#                 elif i == 4:
#                     arrival_rate_SAA[k,t,i] = np.random.poisson(lam = 2)
#                     service_rate_SAA[t,i] = np.random.exponential(scale = 1.02)
#                 else:
#                     arrival_rate_SAA[k,t,i] = np.random.poisson(lam = 1.5)
#                     service_rate_SAA[t,i] = np.random.exponential(scale = 1.01)          
    objvals[b],soln_x_d[b], soln_x_e[b],soln_yt[b] = extensive_formulation_SAA(arrival_rate_SAA, service_rate_SAA, numscen_saa)

lb_mean = np.mean(objvals)
lb_var_tot = 0
for b in range(numbatch):
    lb_var_tot += (objvals[b]-lb_mean)*(objvals[b]-lb_mean)
lb_var = lb_var_tot/(numbatch - 1)
lb_std = math.sqrt(lb_var)
lb_width = lb_std/math.sqrt(numbatch)*2.2621 # t_M-1,alpha/2: from table
print('mean SAA objval of LB part = ', lb_mean)
print ('unbiased stdev of SAA objval of LB part = ', lb_std)
print('objective values of sample of LB part: ',objvals)
print('solutions of sample of LB part - Day Shift Crews: ',soln_x_d)
print('solutions of sample of LB part - Night Shift Crews: ',soln_x_e)

mean SAA objval of LB part =  85912.23475628576
unbiased stdev of SAA objval of LB part =  758.3937000704323
objective values of sample of LB part:  [84886.5956568  84663.26123256 85950.99207969 86951.16048376
 85592.30285    86528.88195238 86870.90662726 86124.78767237
 85785.26712197 85768.19188607]
solutions of sample of LB part - Day Shift Crews:  {0: 4.0, 1: 4.0, 2: 4.0, 3: 4.0, 4: 4.0, 5: 4.0, 6: 4.0, 7: 4.0, 8: 4.0, 9: 4.0}
solutions of sample of LB part - Night Shift Crews:  {0: 2.0, 1: 2.0, 2: 2.0, 3: 2.0, 4: 2.0, 5: 2.0, 6: 2.0, 7: 2.0, 8: 2.0, 9: 2.0}


In [28]:
### here, candx is the solution obtained in last batch above
### more generally, might want to choose "most promising" solution
evalvals = np.zeros(numeval)

best_obj_idx = objvals.argmin()
print('The index of the best sample average objective value: ', best_obj_idx)
cand_xd = soln_x_d[best_obj_idx]
cand_xe = soln_x_e[best_obj_idx]
cand_yt = soln_yt[best_obj_idx] # try 3 for interim
print('candidate solution = ',cand_xd, cand_xe, cand_yt)
rand_idx_ub = 0
arrival_rate_SAA_={}
service_rate_SAA_ = {}
for k in range(numeval):
    rand_idx_ub = random.randint(0,numscen-1)
    for t in range(24):
        for i in range(6):
            arrival_rate_SAA[t,i] = arrival_rate[rand_idx_ub,t,i]
            service_rate_SAA[t,i] = service_rate[t,i]
#             if i == 0:
#                 arrival_rate_SAA[t,i] = np.random.poisson(lam = 1.2)
#                 service_rate_SAA[t,i] = np.random.exponential(scale = 1.52)
#             elif i == 1:
#                 arrival_rate_SAA[t,i] = np.random.poisson(lam = 1.3)
#                 service_rate_SAA[t,i] = np.random.exponential(scale = 1.2)
#             elif i == 2:
#                 arrival_rate_SAA[t,i] = np.random.poisson(lam = 1.2)
#                 service_rate_SAA[t,i] = np.random.exponential(scale = 1.61)
#             elif i == 3:
#                 arrival_rate_SAA[t,i] = np.random.poisson(lam = 1.3)
#                 service_rate_SAA[t,i] = np.random.exponential(scale = 2.1)
#             elif i == 4:
#                 arrival_rate_SAA[t,i] = np.random.poisson(lam = 2)
#                 service_rate_SAA[t,i] = np.random.exponential(scale = 1.02)
#             else:
#                 arrival_rate_SAA[t,i] = np.random.poisson(lam = 1.5)
#                 service_rate_SAA[t,i] = np.random.exponential(scale = 1.01)
    evalvals[k] = SAA_solveSSP(cand_xd, cand_xe, cand_yt, service_rate_SAA, arrival_rate_SAA)
            
print('mean solution objval of UB part = ', np.mean(evalvals))
ub_mean = np.mean(evalvals)
ub_var_tot = 0
for k in range(numeval):
    ub_var_tot += (evalvals[k]-ub_mean)*(evalvals[k]-ub_mean)
ub_var = ub_var_tot/(numeval - 1)
ub_std = math.sqrt(ub_var)
ub_width = ub_std/math.sqrt(numeval)*1.645 # critical z_value: z_alpha, different from t_value
print('unbiased stdev of solution objval of UB part = ', ub_std)

print(' ')
print('95% ci on lower bound = [', lb_mean-lb_width, ',', lb_mean+lb_width, ']')
print('95% ci on upper bound = [', ub_mean-ub_width, ',', ub_mean+ub_width, ']')
print('The worst optimality gap = [', lb_mean-lb_width, ',', ub_mean+ub_width, ']')


The index of the best sample average objective value:  1
candidate solution =  4.0 2.0 {0: 2.0, 1: 2.0, 2: 2.0, 3: 2.0, 4: 2.0, 5: 2.0, 6: 2.0, 7: 4.0, 8: 4.0, 9: 4.0, 10: 4.0, 11: 4.0, 12: 4.0, 13: 4.0, 14: 4.0, 15: 4.0, 16: 4.0, 17: 4.0, 18: 4.0, 19: 2.0, 20: 2.0, 21: 2.0, 22: 2.0, 23: 2.0}
mean solution objval of UB part =  85889.15725840861
unbiased stdev of solution objval of UB part =  7329.7726658711545
 
95% ci on lower bound = [ 85369.72629457213 , 86454.74321799939 ]
95% ci on upper bound = [ 85349.9305373659 , 86428.38397945132 ]
The worst optimality gap = [ 85369.72629457213 , 86428.38397945132 ]
